## Computational Physics 2 (WS23/24) – Warm-up exercise

**Deadline: 31.10.2023 at 23:59**

Group: *write group name*
Students: *write names and matriculation numbers*

You will implement and test two algorithms: **conjugate gradient** and **power method**. We will see in a moment what they are useful for. Fill the notebook following the instructions.

### Initialization

Here we load the needed libraries and we initialize the random number generator. **Important**: when using a random number generator, the seed needs to be set only once, preferebly at the beginning of the program.

In [1]:
import numpy as np

rng = np.random.Generator(np.random.PCG64(12345))

### Positive-definite matrices

Both algorithms will deal with hermitian positive-definite matrices. Recall:

- Given a complex square matrix $A$, its hermitian conjugate $A^\dagger$ is defined as its transposed complex-conjugated, i.e. $(A^\dagger)_{ij} = (A_{ji})^*$.
- A complex square matrix $A$ is said to be hermitian if $A=A^\dagger$.
- An hermitian matrix $A$ is said to be positive-definite if all its eigenvalues are positive.

The following function generates and returns a random positive-definite matrix, along with its eigenvactors and eigenvalues.

In [2]:
# The function 'generate_positive_definite_matrix' contructs an NxN positive-definite matrix 'A',
# its matrix of eigenvectors and its eigenvalues.
#
# Input parameters:
#    N (integer)        : size of output matrix 'A'
#    kappa (double)     : condition number of the output matrix 'A'
#                         see https://en.wikipedia.org/wiki/Condition_number#Matrices
# Output values: (A, U, evalues)
#    A (np.matrix)      : positive-definite NxN matrix with condition number kappa
#    U (np.matrix)      : NxN unitary matrix; each column of 'U' is an eigenvector of 'A'
#    evalues (np.array) : N-component array with eigenvalues of 'A'

def generate_positive_definite_matrix(N,kappa=10.):
    assert isinstance(N, int) and N > 1 , "N=" + str(N) + " must be an integer >= 2"
    assert isinstance(kappa, float) and kappa > 0. , "kappa=" + str(kappa) + " must be a positive float"
    
    rmat = np.asmatrix(rng.standard_normal(size=(N,N)) + 1j * rng.standard_normal(size=(N,N)))
    U , _ = np.linalg.qr(rmat,mode='complete')
    evalues = np.concatenate((1. + kappa*rng.random(N-2),[1.,kappa]))
    D = np.asmatrix(np.diag(evalues))
    A = np.matmul(np.matmul(U,D),U.getH())
    
    return A, U , evalues

### Power method

Given a positive-definite matrix $A$, the power method allows to approximate its largest eigenvalue and the corresponding eigenvector with a certain specified tolerance $\epsilon$. It is an iterative method: a number of steps are repeated cyclically, at each iteration one gets a better approximation of the eigenvalue and eigenvectors, the iteration is stopped when the approximation is good enough. It works as follows:

1. Generate a random complex vector $v$ with norm equal to 1.
2. Calculate $w=Av$ and $\mu = \| w \|$.
3. If $\| w - \mu v \| < \epsilon$, stop iteration and returns $\mu$ and $v$ are eigenvalue and eigenvector.
4. Replace $v \leftarrow \mu^{-1} w$ and repeat from 2.

**Task:** Implement the power method within the function ```power_method```, with the following specifications.

The *vector* $v$ is not necessarily a one-dimensional array, we want the flexibility to use more abstract vector spaces whose elements are generic $d$-dimensional arrays. In practice, the *vectors* $v$ must be implemented as ```numpy.ndarrays```. In this setup, the squared norm $\|v\|^2$ of the *vector* $v$ is given by the sum of the squared absolute value of all elements of $v$. Moreover, the *matrix* $A$ really needs to be thought as a linear function acting on the elements of the abstract vector space.

```power_method``` must be a function that takes three inputs:
- ```vshape``` is the shape of the elements $v$ of the abstract vector space;
- ```apply_A``` is a function that takes the vector $v$ (represented as an instance of ```numpy.ndarrays```) and returns the vector $Av$ (represented as an instance of ```numpy.ndarrays``` with the same shape as $v$);
- ```epsilon``` is the tolerance.

```power_method``` must return:
- the largest eignevalue $\mu$;
- the corresponding eigenvector (represented as an instance of ```numpy.ndarrays``` with the same shape as the input of ```apply_A```);
- the number of iterations.

A test function is provided below. Your implementation of ```power_method``` needs to pass this test.

In [3]:
# The function 'power_method' calculates an approximation of the largest eigenvalue 'mu'
# and corresponding eigenvector 'v' of the positive-definite linear map 'A'. The quality
# of the approximation is dictated by the tolerance 'epsilon', in the sense that the
# approximated eigenvalue and eigenvector satisfy
#   | A(v) - mu*v | < epsilon
#
# The vectors over which 'A' acts are generically d-dimensional arrays. More precisely,
# they are instances of 'numpy.ndarray' with shape 'vshape'.
#
# The linear map 'A' is indirectly provided as a function 'apply_A' which takes a vector
# v and returns the vector A(v).
#
# Input parameters of power_method:
#    vshape (tuple of ints) : shape of the arrays over which 'A' acts
#    apply_A (function)     : function v -> A(v)
#    epsilon (float)        : tolerance
# Output values: (mu, v, niters)
#    mu (float)             : largest eigenvalue of A
#    v (numpy.ndarray)      : corresponding eigenvector
#    niters (int)           : number of iterations

def power_method(vshape,apply_A,epsilon):
    assert callable(apply_A) , "apply_A must be a function"
    assert isinstance(epsilon, float) and epsilon > 0. , "epsilon=" + str(epsilon) + " must be a positive float"
    assert isinstance(vshape,tuple) , "vshape must be a tuple"
    
    ### Implement your function here

    v = rng.random(vshape)
    niters = 0; maxiters = 1e5

    while niters < maxiters:
        w = apply_A(v)
        mu = np.linalg.norm(w)
        if np.linalg.norm(w-mu*v) < epsilon: break
        v = w/mu
        niters += 1
    
    return mu, v, niters

#### Test

Run the following cell. If the power method is correctly implemented, then the test will pass.

In [4]:
def test_power_method():

    def test_engine(shape,epsilon):
        
        N = int(np.prod(shape))
        A , _ , _ = generate_positive_definite_matrix(N)
        
        def apply_A(v):
            assert isinstance(v,np.ndarray) , "v must be an np.ndarray"
            assert v.shape==shape , "v has shape "+str(v.shape)+", it must have shape "+str(shape)
            return np.asarray(np.dot(A,v.flatten())).reshape(shape)
        
        mu , v , niters = power_method(shape,apply_A,epsilon)
        delta = apply_A(v) - mu*v
        res = np.sqrt(np.vdot(delta,delta).real)
        print("shape = " , shape , "\tresidue = " , res , "\titerations = " , niters , "\tTest passes: " , res<=epsilon)
    
    
    test_engine((4,),1.e-8)
    test_engine((1,5),1.e-12)
    test_engine((3,2,4),1.e-8)
    test_engine((5,2),1.e-12)

test_power_method()

shape =  (4,) 	residue =  9.956240253993894e-09 	iterations =  534 	Test passes:  True
shape =  (1, 5) 	residue =  6.114055854552965e-13 	iterations =  34 	Test passes:  True
shape =  (3, 2, 4) 	residue =  9.80057807665688e-09 	iterations =  720 	Test passes:  True
shape =  (5, 2) 	residue =  9.95017459382059e-13 	iterations =  3428 	Test passes:  True


### Conjugate gradient

**Task.**
1. Read about the conjugate gradient on Wikipedia.
2. Implement the conjugate gradient, using same conventions as for the power method.
3. Write a description of the algorithm here (in the same spirit as the description of the power method).
4. Run and pass the test provided below.
5. Discuss intermediate steps with tutors.

In [ ]:
# The function 'conjugate_gradient' calculates an approximation 'x' of 'A^{-1}(b)', where
# 'A' is a positive-definite linear map 'A', and 'b' is a vector in the domain of 'A'.
# The quality of the approximation is dictated by the tolerance 'epsilon', in the sense
# that the following inequality is satisfied
#   | A(x) - b | <= epsilon |b|
#
# The vectors over which 'A' acts are generically d-dimensional arrays. More precisely,
# they are instances of 'numpy.ndarray' with shape 'vshape'.
#
# The linear map 'A' is indirectly provided as a function 'apply_A' which takes a vector
# v and returns the vector A(v).
#
# Input parameters of power_method:
#    apply_A (function)     : function v -> A(v)
#    b (numpy.ndarray)      : vector 'b'
#    epsilon (float)        : tolerance
# Output values: (x, niters)
#    x (numpy.ndarray)      : approximation of 'A^{-1}(b)'
#    niters (int)           : number of iterations

def conjugate_gradient(apply_A,b,epsilon):

    ### Implement your function here
    
    return x, niters

In [5]:
def test_conjugate_gradient():

    def test_engine(shape,epsilon):
        
        N = int(np.prod(shape))
        A , _ , _ = generate_positive_definite_matrix(N)
        b = rng.standard_normal(size=shape) + 1j * rng.standard_normal(size=shape)
        
        def apply_A(v):
            assert isinstance(v,np.ndarray) , "v must be an np.ndarray"
            assert v.shape==shape , "v has shape "+str(v.shape)+", it must have shape "+str(shape)
            return np.asarray(np.dot(A,v.flatten())).reshape(shape)
        
        x , niters = conjugate_gradient(apply_A,b,epsilon)
        delta = apply_A(x) - b
        res = np.sqrt(np.vdot(delta,delta).real)
        print("shape = " , shape , "\tresidue = " , res , "\titerations = " , niters , "\tTest passes: " , res<=epsilon*np.sqrt(np.vdot(b,b).real))
    
    
    test_engine((4,),1.e-8)
    test_engine((1,5),1.e-12)
    test_engine((3,2,4),1.e-8)
    test_engine((5,2),1.e-12)

test_conjugate_gradient()

NameError: name 'conjugate_gradient' is not defined